# Entrenamiento de una RN

Al entrenar una red buscamos que ésta tenga un comportamiendo presumiblemente observable en un conjunto de datos. Es decír a partir del conjunto de ejemplos esperamos que la red aprenda una función $f$ tal que la salida de la red imite el patrón en los datos. En nuestro caso, al utilizar el conjunto de entrenamiento MNIST esperamos que la función aprendida provea como salida el número al cual corresponde la imagen de entrada.

<img src="archivos/function_approx.png">
Imagen tomada de Udacity[]



## Gradientes y Autograd

Pytorch provee el módulo *autograd* para calcular los gradientes, y si, !nos evita estar calculando las derivadas! Esto lo realiza a partir de mantener en la vista todas las operaciones que se hacen a los tensores.

Si deseas asegurarte que autograd siga a un tensor especificamos *requires_grad*. Esto se puede hacer en la creación o en cualquier momento. 

Veamos el siguiente código de ejemplo:

```python
# especificamos que la variable x es seguida por autograd
x = torch.zeros(1, requires_grad=True)
# si en algún momento desamos que temporalmente se deje de seguir el tensor usamos
>>> with torch.no_grad():
...     y = x * 2
# establecemos de nuevo el seguimiento
>>> y.requires_grad
```

Si queremos eliminar autograd de todos los tensores usamos `torch.set_grad_enabled(True|False)`.


Ahora bien, para calcular los gradientes simplemente usamos el método *backward()*. Por ejemplo para un tensor *x* hacemos *z.backward()*

Veamos el uso del gradiente.

In [1]:
# importamos paquetes
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from collections import OrderedDict

import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import helper

In [2]:
# especificamos que el tensor x es seguido por autograd 
x = torch.randn(2,2, requires_grad=True)
print(x)

tensor([[ 0.1440,  1.3263],
        [ 0.5819, -0.1294]], requires_grad=True)


In [3]:
# generamos un nuevo tensor a partir de x
y = x**2
print(y)

tensor([[0.0207, 1.7591],
        [0.3386, 0.0168]], grad_fn=<PowBackward0>)


In [4]:
## con grad_fn observamos la operación que generó y, es decir una operación potencia (pow)
print(y.grad_fn)



In [5]:
# De esta forma es posible saber las operaciónes que generan cada tensor, y por tanto, es posible calcular el gradiente.

# Hagamos ahora una operación de media

z = y.mean()
print(z)

tensor(0.5338, grad_fn=<MeanBackward1>)


In [6]:
# hasta este momento los gradientes son cero

print(x.grad)

None


Para calcular los gradientes es necesario llamar al método *.backward* sobre la variable. Supongamos sobre *z*. Esto calcula el gradiente de z con respecto de x.

El gradiente analítico de las operaciónes que hicimos es:
$$
\frac{\partial z}{\partial x} = \frac{\partial}{\partial x}\left[\frac{1}{n}\sum_i^n x_i^2\right] = \frac{x}{2}
$$

Ahora comprobemos

In [7]:
z.backward()
print(x.grad)

print(x/2)

tensor([[ 0.0720,  0.6632],
        [ 0.2910, -0.0647]])
tensor([[ 0.0720,  0.6632],
        [ 0.2910, -0.0647]], grad_fn=<DivBackward0>)


## Dataset y Red Neuronal

Ahora descargemos los datos y generemos una red tal cual lo vimos en el notebook anterior. 

In [8]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])
# Download and load the training data
trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [9]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('logits', nn.Linear(hidden_sizes[1], output_size))]))

# NOTA solo calcularemos los logits y definiremos la perdida a partir de ellos

## Entrenamiento de la RN

Lo primero que definiremos será la función de pérdida (loss) que es nombrada en pytorch como **criterion**. En este ejemplo estamos utilizando softmax, asi que definimos el criterio como *criterion = nn.CrossEntropyLoss()*. Más tarde, en el entrenamiento, veremos que *loss = criterion(output, targets)* calcula la pérdida.

Lo segundo que definiremos será el optimizador, para este ejemplo usaremos SGD (stochastic gradient descent). Simplemente llamamos a *torch.optim.SGD* y le pasamos los parámetros de la red y el lerning rate. 


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

Antes de relizar el entrenamiento completo haremos un paso del aprendizaje. Este paso se compone de las siguientes tareas:

1. Realizar un pase frontal de la red
2. Utilizar los logits para calcular la pérdida
3. Realizar un pase en reversa para calcular los gradientes.
4. Actualizar los pesos usando el optimizador.

Veamos el ejemplo


In [11]:
print('Initial weights - ', model.fc1.weight)


Initial weights -  Parameter containing:
tensor([[ 0.0225, -0.0183, -0.0109,  ..., -0.0325,  0.0132, -0.0265],
        [ 0.0293,  0.0057, -0.0163,  ..., -0.0111, -0.0273,  0.0343],
        [ 0.0249, -0.0158,  0.0322,  ..., -0.0344,  0.0321, -0.0269],
        ...,
        [-0.0228,  0.0120, -0.0271,  ..., -0.0050,  0.0291,  0.0337],
        [-0.0322,  0.0170, -0.0128,  ..., -0.0277,  0.0251,  0.0003],
        [-0.0151,  0.0169,  0.0311,  ...,  0.0344, -0.0242, -0.0240]],
       requires_grad=True)


In [12]:
images, labels = next(iter(trainloader))
images.resize_(64, 784)

# Limpiar los gradientes por que se acumulan
optimizer.zero_grad()

# Pase hacia adelante
output = model.forward(images)
# Perdida
loss = criterion(output, labels)
# Pase de reversa
loss.backward()
print('Gradient -', model.fc1.weight.grad)
# Actualiza los pesos de acuerdo a un paso del optimizador
optimizer.step()

Gradient - tensor([[-0.0013, -0.0013, -0.0013,  ..., -0.0013, -0.0013, -0.0013],
        [ 0.0017,  0.0017,  0.0017,  ...,  0.0017,  0.0017,  0.0017],
        [ 0.0001,  0.0001,  0.0001,  ...,  0.0001,  0.0001,  0.0001],
        ...,
        [-0.0004, -0.0004, -0.0004,  ..., -0.0004, -0.0004, -0.0004],
        [-0.0005, -0.0005, -0.0005,  ..., -0.0005, -0.0005, -0.0005],
        [ 0.0042,  0.0042,  0.0042,  ...,  0.0042,  0.0042,  0.0042]])


In [13]:
print('Updated weights - ', model.fc1.weight)

Updated weights -  Parameter containing:
tensor([[ 0.0225, -0.0183, -0.0108,  ..., -0.0325,  0.0132, -0.0265],
        [ 0.0293,  0.0057, -0.0163,  ..., -0.0112, -0.0273,  0.0343],
        [ 0.0249, -0.0158,  0.0322,  ..., -0.0344,  0.0321, -0.0269],
        ...,
        [-0.0228,  0.0120, -0.0271,  ..., -0.0050,  0.0291,  0.0337],
        [-0.0322,  0.0170, -0.0128,  ..., -0.0277,  0.0251,  0.0003],
        [-0.0151,  0.0168,  0.0311,  ...,  0.0344, -0.0243, -0.0240]],
       requires_grad=True)


## Entrenamiento por épocas

Ahora si, entrenemos la red por varias épocas. 

In [14]:
# configuración del optimizador
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [15]:
# hiperparámetro: número de épocas
epochs = 3
print_every = 40
steps = 0
for e in range(epochs):
    running_loss = 0
    # en cada iteración del for cargamos un batch
    for images, labels in iter(trainloader):
        steps += 1
        # Flatten MNIST images into a 784 long vector
        images.resize_(images.size()[0], 784)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        # actualizamos los pesos
        optimizer.step()
        
        running_loss += loss.item()
        # imprimimos cada 40 batches
        if steps % print_every == 0:
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every))
            
            running_loss = 0

Epoch: 1/3...  Loss: 2.3043
Epoch: 1/3...  Loss: 2.2875
Epoch: 1/3...  Loss: 2.2668
Epoch: 1/3...  Loss: 2.2503
Epoch: 1/3...  Loss: 2.2288
Epoch: 1/3...  Loss: 2.2081
Epoch: 1/3...  Loss: 2.1830
Epoch: 1/3...  Loss: 2.1528
Epoch: 1/3...  Loss: 2.1213
Epoch: 1/3...  Loss: 2.0905
Epoch: 1/3...  Loss: 2.0570
Epoch: 1/3...  Loss: 2.0129
Epoch: 1/3...  Loss: 1.9700
Epoch: 1/3...  Loss: 1.9250
Epoch: 1/3...  Loss: 1.8609
Epoch: 1/3...  Loss: 1.8056
Epoch: 1/3...  Loss: 1.7367
Epoch: 1/3...  Loss: 1.6843
Epoch: 1/3...  Loss: 1.6139
Epoch: 1/3...  Loss: 1.5398
Epoch: 1/3...  Loss: 1.4718
Epoch: 1/3...  Loss: 1.3979
Epoch: 1/3...  Loss: 1.3337
Epoch: 2/3...  Loss: 0.6942
Epoch: 2/3...  Loss: 1.2206
Epoch: 2/3...  Loss: 1.1915
Epoch: 2/3...  Loss: 1.0995
Epoch: 2/3...  Loss: 1.0551
Epoch: 2/3...  Loss: 1.0209
Epoch: 2/3...  Loss: 0.9777
Epoch: 2/3...  Loss: 0.9203
Epoch: 2/3...  Loss: 0.8960
Epoch: 2/3...  Loss: 0.8792
Epoch: 2/3...  Loss: 0.8299
Epoch: 2/3...  Loss: 0.8160
Epoch: 2/3...  Loss:

Finalmente, veamos que tan bien está clasificando la red

In [16]:
images, labels = next(iter(trainloader))

img = images[0].view(1, 784)
# Turn off gradients to speed up this part
with torch.no_grad():
    logits = model.forward(img)

# Output of the network are logits, need to take softmax for probabilities
ps = F.softmax(logits, dim=1)
helper.view_classify(img.view(1, 28, 28), ps)

AttributeError: module 'helper' has no attribute 'view_classify'